In [1]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

# <font color=4CBB17>**3. Sistema de recomendación basado en contenido KNN, con base en todo lo visto por el usuario**</font>

# <font color=4CBB17>**4. Sistemas de recomendación basados en filtro colaborativo**</font>

In [3]:
#instalar lightfm para sistema de recomendación de filtro colaborativo


!pip install lightfm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=825340 sha256=e0a25b675cd874a905d741cfe0d077de4c7eb3442cea54362f2a72f6bbd588c4
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [6]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import a_funciones as fn
from sklearn.preprocessing import MinMaxScaler
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

In [22]:
# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [23]:
ratings = pd.read_sql('SELECT * FROM full_ratings ', con)
ratings

,movie_id,user_id,movie_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_title,movie_year
0,1,1,4.0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,Toy Story,1995
1,3,1,4.0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,Grumpier Old Men,1995
2,6,1,4.0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,Heat,1995
3,47,1,5.0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,Seven (a.k.a. Se7en),1995
4,50,1,5.0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,"Usual Suspects, The",1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36762,592,609,3.0,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,Batman,1989
36763,742,609,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Thinner,1996
36764,786,609,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,Eraser,1996
36765,892,609,3.0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,Twelfth Night,1996


In [14]:
#from sklearn.preprocessing import MinMaxScaler

# Seleccionamos solo las columnas que vamos a escalar
#colums = ['movie_year', 'movie_rating']

In [16]:
#scaler = MinMaxScaler()

# Ajustar y transformar las columnas
#ratings[colums] = scaler.fit_transform(ratings[colums])

In [24]:

#Crear data set en formato lightfm, primero lista de usuarios unicos y items únicos
dataset_train = Dataset()
dataset_test = Dataset()

all_unique_users = ratings['user_id'].unique()
all_unique_items = ratings['movie_id'].unique()

dataset_train.fit(users=all_unique_users, items=all_unique_items)
dataset_test.fit(users=all_unique_users, items=all_unique_items)

In [25]:
## verificar que el data set creado tenga los mismos registros que base original
cur.execute('SELECT count(distinct user_id), count (distinct movie_id) FROM full_ratings ')
print(cur.fetchall())

# Se compara la vbase de datos original con la de dataset train
num_users, num_items = dataset_train.interactions_shape()

print(num_users, num_items)


[(414, 2111)]
414 2111


In [29]:
# Separar tarin y test
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=123)

# Para trabajar con el paquete light FM, se deben crear datos que crean consecutivos para cada usuario e item

train_interactions_list = [(row['user_id'], row['movie_id'], row['movie_rating']) for index, row in train_df.iterrows()]

train_interactions, train_weights = dataset_train.build_interactions(train_interactions_list)

test_interactions_list = [(row['user_id'], row['movie_id'], row['movie_rating']) for index, row in test_df.iterrows()]
test_interactions, test_weights = dataset_test.build_interactions(test_interactions_list)

print(f"Shape of Interactions Matrix: {train_interactions.shape}")
print(f"Shape of Weights Matrix: {train_weights.shape}")
print(f"Shape of Interactions Matrix: {test_interactions.shape}")
print(f"Shape of Weights Matrix: {test_weights.shape}")

Shape of Interactions Matrix: (414, 2111)
Shape of Weights Matrix: (414, 2111)
Shape of Interactions Matrix: (414, 2111)
Shape of Weights Matrix: (414, 2111)


In [30]:
#Este código es de verificación para mirar los datos cómo se estructuran
train_int_array=train_interactions.toarray()
train_weights_array=train_weights.toarray()

test_int_array=test_interactions.toarray()
test_weights_array=test_weights.toarray()

print(train_int_array)
print(test_int_array)

print(train_weights_array)
print(test_weights_array)

[[1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]]
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[4. 0. 4. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 4.]
 [3. 0. 0. ... 0. 0. 0.]]
[[0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 4. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:

import time

############# ajuste manual del modelo
loss_model='logistic'  ## 'logistic', 'bpr', 'warp', 'warp-kos': este último solo sirve para implicitas

model=LightFM(loss=loss_model, random_state=123)
model.fit(train_interactions, epochs=10, verbose=True, sample_weight=train_weights)

### auc por usuario
train_auc = auc_score(model, train_interactions)
test_auc = auc_score(model, test_interactions)


print('AUC: train %.2f, test %.2f.' % (train_auc.mean(), test_auc.mean()))



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.4 MB/s eta 0:00:00


Epoch: 100%|██████████| 10/10 [00:00<00:00, 104.99it/s]


AUC: train 0.82, test 0.78.


In [ ]:
!pip install --quiet optuna
import optuna

#### para afinar hiperparámetros con optuna

def objective(trial):

    # grilla de hiperparámetros
    n_components = trial.suggest_int('no_components', 10, 100)       # Sugiere un entero para el número de componentes latentes (dimensionalidad del modelo).
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True) # Sugiere un flotante (en escala logarítmica) para la tasa de aprendizaje del optimizador.
    loss = trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp']) # Sugiere un valor categórico para la función de pérdida ('logistic', 'bpr' o 'warp').
    epochs = trial.suggest_int('epochs', 10, 50)                     # Sugiere un entero para el número de épocas de entrenamiento.

    print(f"  Params: n_components={n_components}, lr={learning_rate:.4f}, loss={loss},  epochs={epochs}")

    # Modelo con parámetros sugeridos
    model = LightFM(
        no_components=n_components,
        learning_rate=learning_rate,
        loss=loss,

        random_state=42
    )


    model.fit(train_interactions,
              epochs=epochs,
              verbose=False,
              sample_weight=train_weights)

    test_auc = auc_score(model, test_interactions).mean()



    return test_auc


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # You can adjust the number of trials

#Resultados
print(f"Best trial:")
print(f"  Value (Test AUC): {study.best_value:.4f}")
print(f"  Params: {study.best_params}")

#Entrenar modelo con mejores hyperparametros
best_params = study.best_params


model = LightFM(
    no_components=best_params['no_components'],
    learning_rate=best_params['learning_rate'],
    loss=best_params['loss'],
    random_state=42
)


model.fit(train_interactions,
                epochs=best_params['epochs'],
                verbose=True, # Set to True to see progress
                sample_weight=train_weights)


### resultado final
train_auc = auc_score(model, train_interactions).mean()
test_auc = auc_score(model, test_interactions).mean()

print('Final Model AUC: train %.4f, test %.4f.' % (train_auc, test_auc))


In [72]:
def recommendation(model, data, original_user_id, conn,k):

  #data=dataset_train
  #original_user_id=52853

  ## libros no leidos y códigos
  df_nr_movies=pd.read_sql(f'select * from full_ratings where movie_id<>{original_user_id}',conn)
  id_nr_movie=df_nr_movies['movie_id'].values
  item_id_nr=[value for key, value in data.mapping()[2].items() if value not in id_nr_movie]

  uid_index=data.mapping()[0][original_user_id] ## id usuario según modelo

  scores = model.predict(uid_index, item_id_nr)
  sorted_indices = np.argsort(-scores).tolist()

  top_items = [key for key, value in data.mapping()[2].items() if value in sorted_indices[:k]]
  recommended=df_nr_movies[df_nr_movies['movie_id'].isin(top_items)][['movie_id', 'movie_title']]
  recommended.drop_duplicates(inplace=True)

  return recommended

In [74]:
recommendation(model, dataset_train, 100
               , con, 10)

,movie_id,movie_title
0,1,Toy Story
1,3,Grumpier Old Men
4,50,"Usual Suspects, The"
6,101,Bottle Rocket
25,480,Jurassic Park
82,1291,Indiana Jones and the Last Crusade
90,1552,Con Air
190,3439,Teenage Mutant Ninja Turtles II: The Secret of...
233,21,Get Shorty
240,162,Crumb


In [88]:
dataset_train.mapping()[0]

{np.int64(1): 0,
 np.int64(3): 1,
 np.int64(4): 2,
 np.int64(5): 3,
 np.int64(7): 4,
 np.int64(8): 5,
 np.int64(9): 6,
 np.int64(10): 7,
 np.int64(11): 8,
 np.int64(12): 9,
 np.int64(13): 10,
 np.int64(14): 11,
 np.int64(15): 12,
 np.int64(16): 13,
 np.int64(17): 14,
 np.int64(20): 15,
 np.int64(22): 16,
 np.int64(23): 17,
 np.int64(24): 18,
 np.int64(27): 19,
 np.int64(29): 20,
 np.int64(30): 21,
 np.int64(31): 22,
 np.int64(32): 23,
 np.int64(33): 24,
 np.int64(34): 25,
 np.int64(36): 26,
 np.int64(38): 27,
 np.int64(39): 28,
 np.int64(40): 29,
 np.int64(41): 30,
 np.int64(43): 31,
 np.int64(44): 32,
 np.int64(46): 33,
 np.int64(47): 34,
 np.int64(48): 35,
 np.int64(52): 36,
 np.int64(54): 37,
 np.int64(56): 38,
 np.int64(58): 39,
 np.int64(59): 40,
 np.int64(61): 41,
 np.int64(63): 42,
 np.int64(65): 43,
 np.int64(67): 44,
 np.int64(69): 45,
 np.int64(70): 46,
 np.int64(71): 47,
 np.int64(72): 48,
 np.int64(73): 49,
 np.int64(74): 50,
 np.int64(75): 51,
 np.int64(76): 52,
 np.int64(

In [84]:
def recommendation(model, data, original_user_id, conn, k):

    # Cargar todas las películas no vistas por el usuario
    df_nr_movies = pd.read_sql(f'''
        SELECT * FROM full_ratings
        -- Devuelve todos los movie_id que el usuario ya ha calificado, es decir, películas que ya ha visto
        WHERE movie_id <> {original_user_id}''', conn)

    # Obtener los ids de esas películas
    id_nr_movie = df_nr_movies['movie_id'].values

    # Obtener el mapeo de ítems
    item_mapping = data.mapping()[2]  # movie_id -> item_index

    # Obtener los índices internos (LightFM) de las películas no vistas
    item_id_nr = [value for key, value in item_mapping.items() if key in id_nr_movie]

    # Obtener el índice del usuario según el modelo
    uid_index = data.mapping()[0][original_user_id]

    # Predecir scores
    scores = model.predict(uid_index, item_id_nr)
    sorted_indices = np.argsort(-scores)

    # Seleccionar los top-k
    top_k_item_indices = [item_id_nr[i] for i in sorted_indices[:k]]

    # Invertir mapeo para obtener movie_id desde índices
    inverse_item_mapping = {v: k for k, v in item_mapping.items()}
    top_movie_ids = [inverse_item_mapping[i] for i in top_k_item_indices]

    # Mostrar los títulos recomendados
    recommended = df_nr_movies[df_nr_movies['movie_id'].isin(top_movie_ids)][['movie_id', 'movie_title']].drop_duplicates()

    return recommended

In [90]:
recommendation(model, dataset_train, 609, con, 10)

,movie_id,movie_title
3,47,Seven (a.k.a. Se7en)
7,110,Braveheart
16,296,Pulp Fiction
20,356,Forrest Gump
25,480,Jurassic Park
27,527,Schindler's List
33,593,"Silence of the Lambs, The"
418,150,Apollo 13
430,318,"Shawshank Redemption, The"
446,589,Terminator 2: Judgment Day


In [ ]:
import nbformat

# Path to your notebook
input_notebook = 'd_modelos2.ipynb'
output_notebook = 'd_modelos22.ipynb'

# Load the notebook
with open(input_notebook, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# Check if the notebook has 'metadata.widgets' and remove it
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']  # Removes widgets metadata completely

# Alternatively, if you want to add a 'state' key inside 'widgets', do this:
# if 'widgets' in notebook.metadata:
#     notebook.metadata['widgets']['state'] = {}

# Save the modified notebook
with open(output_notebook, 'w') as f:
    nbformat.write(notebook, f)

print(f"Fixed notebook saved as {output_notebook}")